In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

project_root = "/home/bobby/repos/latent-neural-dynamics-modeling"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Exploratory Data Analysis


In [4]:
from pathlib import Path
import polars as pl

from utils.file_handling import get_child_subchilds_tuples
from utils.plots import plot_trial_channel, plot_trial_coordinates, plot_tracing_speed

ModuleNotFoundError: No module named 'polars'

In [4]:
ROOT_PATH = Path(project_root)
DATA_PATH = ROOT_PATH / "resampled_recordings"
PARTICIPANTS_PATH = DATA_PATH / "participants"

In [5]:
def load_participant_session_data(participant_id: str, session_id: str) -> pl.DataFrame:
    p_partition_path = PARTICIPANTS_PATH / participant_id / session_id / "*"
    print(f"Loading data from: {p_partition_path}")
    return pl.read_parquet(p_partition_path)

In [6]:
participants_partitions = get_child_subchilds_tuples(PARTICIPANTS_PATH)
participants_partitions

[('participant_id=PDI4', 'session=3'),
 ('participant_id=PDI4', 'session=2'),
 ('participant_id=PDI1', 'session=4'),
 ('participant_id=PDI1', 'session=2'),
 ('participant_id=PDI3', 'session=4'),
 ('participant_id=PDI3', 'session=2'),
 ('participant_id=PDI2', 'session=4'),
 ('participant_id=PDI2', 'session=2')]

In [7]:
participant_id = "participant_id=PDI1"
session_id = "session=2"

In [21]:
participants = load_participant_session_data(participant_id, session_id)

Loading data from: /home/bobby/repos/latent-neural-dynamics-modeling/resampled_recordings/participants/participant_id=PDI1/session=2/*


In [22]:
participants

participant_id,session,block,trial,onset,margined_onset,margined_duration,time,time_original,motion_time,original_length_ts,start_ts,trial_length_ts,chunk_margin,dbs_stim,LFP_1,LFP_2,LFP_3,LFP_4,LFP_5,LFP_6,LFP_7,LFP_8,LFP_9,LFP_10,LFP_11,LFP_12,LFP_13,LFP_14,LFP_15,LFP_16,LFP_1_epochs,LFP_2_epochs,LFP_3_epochs,LFP_4_epochs,LFP_5_epochs,LFP_6_epochs,…,LFP_8_psd_values,LFP_9_psd_freq,LFP_9_psd_values,LFP_10_psd_freq,LFP_10_psd_values,LFP_11_psd_freq,LFP_11_psd_values,LFP_12_psd_freq,LFP_12_psd_values,LFP_13_psd_freq,LFP_13_psd_values,LFP_14_psd_freq,LFP_14_psd_values,LFP_15_psd_freq,LFP_15_psd_values,LFP_16_psd_freq,LFP_16_psd_values,ECOG_1,ECOG_2,ECOG_3,ECOG_4,ECOG_1_epochs,ECOG_2_epochs,ECOG_3_epochs,ECOG_4_epochs,ECOG_1_psd_freq,ECOG_1_psd_values,ECOG_2_psd_freq,ECOG_2_psd_values,ECOG_3_psd_freq,ECOG_3_psd_values,ECOG_4_psd_freq,ECOG_4_psd_values,x,y,tracing_speed,tracing_coordinates_present
str,u32,u32,u32,f64,f64,f32,list[f64],list[f64],list[f64],u32,u32,u32,i32,i32,list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[list[f64]],list[list[f64]],list[list[f64]],list[list[f64]],list[list[f64]],list[list[f64]],…,list[list[f64]],list[f64],list[list[f64]],list[f64],list[list[f64]],list[f64],list[list[f64]],list[f64],list[list[f64]],list[f64],list[list[f64]],list[f64],list[list[f64]],list[f64],list[list[f64]],list[f64],list[list[f64]],list[f32],list[f32],list[f32],list[f32],list[list[f64]],list[list[f64]],list[list[f64]],list[list[f64]],list[f64],list[list[f64]],list[f64],list[list[f64]],list[f64],list[list[f64]],list[f64],list[list[f64]],list[i64],list[i64],list[f64],bool
"""PDI1""",2,7,1,28.276182,26.276182,13.017329,"[26.276182, 26.277182, … 39.292182]","[28.276182, 28.277182, … 37.292182]","[28.276182, 28.284641, … 37.285052]",9017,13017,13017,2,0,"[0.000011, 0.00001, … 0.000012]","[0.000009, 0.000008, … -0.000006]","[-0.000009, -0.000017, … 0.000051]","[-0.00008, -0.000151, … 0.000249]","[-0.000003, -0.000004, … 0.000013]","[-0.00001, -0.000006, … 0.000086]","[0.000007, 0.000009, … 0.000073]","[0.000009, 0.000009, … 0.000004]","[-0.000027, -0.000023, … 0.000006]","[0.000009, 0.000011, … -0.000002]","[-0.000039, -0.000035, … 0.000003]","[0.000031, 0.000034, … 0.000009]","[-0.000005, -0.000003, … -0.000002]","[0.000028, 0.00003, … 0.000004]","[0.000093, 0.000093, … 0.000009]","[0.000022, 0.000023, … 0.000008]","[[0.000011, 0.00001, … -0.000006], [1.2193e-7, 0.000016, … -8.5874e-7], … [-0.000004, -0.000007, … 0.0]]","[[0.000009, 0.000008, … 0.000051], [-0.000003, 0.000012, … 0.000002], … [-0.000012, -0.000017, … 0.0]]","[[-0.000009, -0.000017, … 0.000006], [0.000037, 0.000045, … 0.000027], … [-0.000059, -0.000063, … 0.0]]","[[-0.00008, -0.000151, … -0.000138], [-0.000168, -0.000174, … -0.000081], … [0.000111, -0.00003, … 0.0]]","[[-0.000003, -0.000004, … 0.000005], [0.000006, 0.000019, … -0.000008], … [2.3753e-7, -0.000002, … 0.0]]","[[-0.00001, -0.000006, … 0.000022], [0.000038, 0.000029, … 0.000047], … [-0.000061, -0.000052, … 0.0]]",…,"[[1.4855e-11, 2.0101e-12, … 2.3594e-13], [3.1103e-11, 2.0426e-11, … 8.2267e-14], … [3.5930e-18, 3.8265e-18, … 9.7743e-18]]","[3.0, 4.0, … 250.0]","[[9.6557e-11, 7.3297e-11, … 2.7553e-13], [5.0118e-11, 2.4771e-11, … 7.8694e-14], … [1.3873e-16, 1.3841e-16, … 8.1755e-18]]","[3.0, 4.0, … 250.0]","[[1.6028e-9, 5.1888e-10, … 2.8698e-13], [2.4219e-9, 2.2280e-10, … 5.4541e-14], … [1.4590e-16, 1.4439e-16, … 5.9434e-18]]","[3.0, 4.0, … 250.0]","[[5.9373e-11, 1.7321e-10, … 2.4395e-13], [3.3767e-10, 2.8140e-10, … 4.8154e-14], … [1.7932e-17, 1.7743e-17, … 6.7563e-18]]","[3.0, 4.0, … 250.0]","[[1.2355e-10, 3.3788e-11, … 2.5578e-13], [5.1087e-10, 1.0814e-10, … 4.9668e-14], … [8.7271e-17, 8.7223e-17, … 1.0948e-17]]","[3.0, 4.0, … 250.0]","[[4.0416e-10, 3.0876e-11, … 2.7312e-13], [8.8154e-10, 2.9791e-10, … 5.5500e-14], … [4.3330e-17, 4.2759e-17, … 5.1073e-18]]","[3.0, 4.0, … 250.0]","[[5.2620e-10, 2

In [9]:
from utils.polars import get_trial

In [12]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=12,
    columns=[
        "time",
        "LFP_1",
        "chunk_margin",
        "margined_duration",
        "dbs_stim",
        "participant_id",
        "trial",
    ],
    explode=["time", "LFP_1"],
)

In [13]:
plot_trial_channel(
    trial_df,
    channel="LFP_1",
)

In [13]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=1,
    columns=[
        "time",
        "ECOG_1",
        "chunk_margin",
        "margined_duration",
        "dbs_stim",
        "participant_id",
        "trial",
    ],
    explode=["time", "ECOG_1"],
)

In [14]:
plot_trial_channel(
    trial_df,
    channel="ECOG_1",
)

* **Visualize Recordings with DBS Markers:**

In [14]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=1,
    columns=[
        "motion_time",
        "x",
        "y",
        "chunk_margin",
        "margined_duration",
        "dbs_stim",
        "participant_id",
        "trial",
        "session",
        "block",
    ],
    explode=["motion_time", "x", "y"],
)

In [15]:
plot_trial_coordinates(
    trial_df,
    time="motion_time",
)

In [17]:
trial_df.head(1)

motion_time,x,y,chunk_margin,margined_duration,dbs_stim,participant_id,trial,session,block
f64,i64,i64,i32,f32,i32,str,u32,u32,u32
18.102864,-377,37,2,13.019526,0,"""PDI1""",1,2,1


In [16]:
plot_trial_coordinates(
    trial_df,
    time="motion_time",
    plot_over_time=True,
)

In [19]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=1,
    columns=[
        "time_original",
        "x_interpolated",
        "y_interpolated",
        "chunk_margin",
        "margined_duration",
        "dbs_stim",
        "participant_id",
        "trial",
        "session",
        "block",
    ],
    explode=["time_original", "x_interpolated", "y_interpolated"],
)

ColumnNotFoundError: unable to find column "x_interpolated"; valid columns: ["participant_id", "session", "block", "trial", "onset", "margined_onset", "margined_duration", "time", "time_original", "motion_time", "original_length_ts", "start_ts", "trial_length_ts", "chunk_margin", "dbs_stim", "yscore", "LFP_1", "LFP_2", "LFP_3", "LFP_4", "LFP_5", "LFP_6", "LFP_7", "LFP_8", "LFP_9", "LFP_10", "LFP_11", "LFP_12", "LFP_13", "LFP_14", "LFP_15", "LFP_16", "LFP_1_epochs", "LFP_2_epochs", "LFP_3_epochs", "LFP_4_epochs", "LFP_5_epochs", "LFP_6_epochs", "LFP_7_epochs", "LFP_8_epochs", "LFP_9_epochs", "LFP_10_epochs", "LFP_11_epochs", "LFP_12_epochs", "LFP_13_epochs", "LFP_14_epochs", "LFP_15_epochs", "LFP_16_epochs", "LFP_1_freq", "LFP_1_values", "LFP_2_freq", "LFP_2_values", "LFP_3_freq", "LFP_3_values", "LFP_4_freq", "LFP_4_values", "LFP_5_freq", "LFP_5_values", "LFP_6_freq", "LFP_6_values", "LFP_7_freq", "LFP_7_values", "LFP_8_freq", "LFP_8_values", "LFP_9_freq", "LFP_9_values", "LFP_10_freq"

In [ ]:
plot_trial_coordinates(
    trial_df, x="x_interpolated", y="y_interpolated", time="time_original"
)

In [ ]:
plot_trial_coordinates(
    trial_df,
    x="x_interpolated",
    y="y_interpolated",
    time="time_original",
    plot_over_time=True,
)

In [ ]:
from utils.polars import read_tsv

In [ ]:
sample_participant = "/home/bobby/repos/latent-neural-dynamics-modeling/data/sub-PDI1/ses-2/motion/sub-PDI1_ses-2_task-copydraw_run-01_chunk-01_tracksys-wacom_motion.tsv"

In [ ]:
df = read_tsv(sample_participant, unique=False)

In [ ]:
df.head(1)

x,y
i64,i64
-377,37


In [ ]:
plot_trial_coordinates(df)

### 3. Power Spectral Density (PSD) Analysis (for RQ2)

* **Segment Data:** Separate the preprocessed data into epochs corresponding to DBS ON and DBS OFF conditions.
* **Calculate PSD:**
    - Compute PSD estimates for LFP channels during DBS ON vs. OFF.
    - Compute PSD estimates for ECoG channels during DBS ON vs. OFF.
* **Compare PSDs:**
    - Plot the mean PSD for ON vs. OFF conditions for both LFP and ECoG to identify potential neural markers (e.g., changes in beta-band power).
* **Statistical Testing:**
    - Implement or use a library function for the cluster-based permutation test.
    - Run the test on the PSD data to find significant differences between DBS ON and OFF states.


In [20]:
from utils.ieeg import epoch_trials, calculate_psd_multitaper

In [21]:
participants = participants.with_columns(
    pl.col("ECOG_1")
    .map_elements(lambda x: epoch_trials(x), return_dtype=pl.List(pl.List(pl.Float64)))
    .alias("ECOG1_epochs")
)

In [22]:
participants = participants.with_columns(
    pl.col("ECOG1_epochs")
    .map_elements(calculate_psd_multitaper, return_dtype=pl.Object)
    .alias("ECOG1_psd")
).with_columns(
    pl.col("ECOG1_psd")
    .map_elements(lambda x: x[0], return_dtype=pl.List(pl.Float64))
    .alias("ECOG1_freq"),
    pl.col("ECOG1_psd")
    .map_elements(lambda x: x[1], return_dtype=pl.List(pl.List(pl.Float64)))
    .alias("ECOG1_values"),
)

    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.

    Using multitaper spectrum estimation with 3 DPSS windows


    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows


[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    1.2s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    1.2s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    1.3s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    1.3s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    1.3s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    1.3s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    1.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    1.3s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    1.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    1.3s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    1.3s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   5 out of

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:   

    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows


[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:   

    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows
    Using multitaper spectrum estimation with 3 DPSS windows


[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    0.2s fi

In [23]:
participants_dbs_on = participants.filter(pl.col("dbs_stim") == 1)

In [24]:
participants_dbs_off = participants.filter(pl.col("dbs_stim") == 0)

In [25]:
from utils.plots import plot_psd_heatmap, plot_avg_psd

In [26]:
import numpy as np

In [ ]:
channels = participants_dbs_on[20].select(pl.col("^ECOG1_.*$")).to_numpy()[0]

In [ ]:
channels = np.array([ch for ch in channels], dtype=np.float64).astype(np.float64)

In [ ]:
channels.shape

(16, 13018)

In [ ]:
channels.shape

(16, 13018)

In [ ]:
from sklearn.decomposition import FastICA

In [ ]:
ica = FastICA(
    n_components=16,
    random_state=97,
    whiten="unit-variance",
    max_iter=1000,
    whiten_solver="eigh",
)
sources = ica.fit_transform(channels)

/home/bobby/miniconda3/envs/neuro/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:615: UserWarning:

There are some small singular values, using whiten_solver = 'svd' might lead to more accurate results.



In [ ]:
artifact_freq = 130.0
freq_band = (artifact_freq - 5, artifact_freq + 5)

In [ ]:
from scipy.signal import welch

In [ ]:
max_power_per_component = []
# Note: sources are [n_samples, n_components], so we iterate over columns
for i in range(sources.shape[1]):
    component_ts = sources[:, i]
    freqs, psd = welch(component_ts, fs=1000, nperseg=int(1000 * 2))

    band_indices = np.where((freqs >= freq_band[0]) & (freqs <= freq_band[1]))[0]

    max_power = np.max(psd[band_indices]) if len(band_indices) > 0 else 0
    max_power_per_component.append(max_power)

# The artifact component is the one with the highest power
artifact_component_index = np.argmax(max_power_per_component)
print(f"Artifact identified in component number: {artifact_component_index}")

# --- Step 4: Reconstruct the signal without the artifact ---

# We nullify (set to zero) the artifact component
sources[:, artifact_component_index] = 0.0

# Use the inverse transform to remix the sources back to channels
cleaned_data_transposed = ica.inverse_transform(sources)

# Transpose back to our original shape [n_channels, n_samples]
cleaned_data = cleaned_data_transposed.T

Artifact identified in component number: 8


/tmp/ipykernel_49008/3128366693.py:5: UserWarning:

nperseg=2000 is greater than signal length max(len(x), len(y)) = 16, using nperseg = 16



In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
time = np.arange(channels.shape[1])

In [ ]:
n_components = sources.shape[1]

# Loop through each component and create a plot
for i in range(n_components):

    # Create a figure with 1 row and 2 columns for the subplots
    fig = make_subplots(
        rows=1, cols=2, subplot_titles=("Time Series (first 3s)", "Power Spectrum")
    )

    # --- Plot 1: Time Series ---
    component_ts = sources[:, i]
    fig.add_trace(
        go.Scatter(
            x=time[: 1000 * 3],
            y=component_ts[: 1000 * 3],
            mode="lines",
            name="Time Series",
        ),
        row=1,
        col=1,
    )

    # --- Plot 2: Power Spectral Density (PSD) ---
    freqs, psd = welch(component_ts, fs=1000, nperseg=1000 * 2)
    fig.add_trace(go.Scatter(x=freqs, y=psd, mode="lines", name="PSD"), row=1, col=2)

    # --- Update plot layout and axes ---
    fig.update_layout(title_text=f"Component {i}", showlegend=False)
    fig.update_xaxes(title_text="Time (s)", row=1, col=1)
    fig.update_yaxes(title_text="Amplitude", row=1, col=1)

    fig.update_xaxes(title_text="Frequency (Hz)", range=[0, 1000 / 2], row=1, col=2)
    fig.update_yaxes(
        title_text="Power (log scale)", type="log", row=1, col=2
    )  # Use log scale

    # Show the interactive plot
    fig.show()

/tmp/ipykernel_49008/24910578.py:20: UserWarning:

nperseg=2000 is greater than signal length max(len(x), len(y)) = 16, using nperseg = 16



In [27]:
first_p_on = participants_dbs_on[20].select(
    "participant_id", "session", "block", "trial", "ECOG1_freq", "ECOG1_values"
)

In [28]:
first_p_on

participant_id,session,block,trial,ECOG1_freq,ECOG1_values
str,u32,u32,u32,list[f64],list[list[f64]]
"""PDI1""",2,4,9,"[3.0, 3.5, … 250.0]","[[1.6302e-8, 1.8813e-8, … 1.0317e-11], [8.5108e-10, 1.4656e-9, … 5.8382e-12], … [2.0059e-12, 1.9820e-12, … 5.8245e-13]]"


In [29]:
plot_psd_heatmap(
    np.array(first_p_on["ECOG1_freq"][0]),
    np.array([np.array(arr) for arr in first_p_on["ECOG1_values"][0]]),
)

In [30]:
plot_avg_psd(
    np.array(first_p_on["ECOG1_freq"][0]),
    np.array([np.array(arr) for arr in participants_dbs_on[0]["ECOG1_values"][0]]),
)

In [31]:
first_p_off = participants_dbs_off[20].select(
    "participant_id", "session", "block", "trial", "LFP1_freq", "LFP1_values"
)

ColumnNotFoundError: unable to find column "LFP1_freq"; valid columns: ["participant_id", "session", "block", "trial", "onset", "margined_onset", "margined_duration", "time", "time_original", "motion_time", "original_length_ts", "start_ts", "trial_length_ts", "chunk_margin", "dbs_stim", "yscore", "LFP_1", "LFP_2", "LFP_3", "LFP_4", "LFP_5", "LFP_6", "LFP_7", "LFP_8", "LFP_9", "LFP_10", "LFP_11", "LFP_12", "LFP_13", "LFP_14", "LFP_15", "LFP_16", "LFP_1_epochs", "LFP_2_epochs", "LFP_3_epochs", "LFP_4_epochs", "LFP_5_epochs", "LFP_6_epochs", "LFP_7_epochs", "LFP_8_epochs", "LFP_9_epochs", "LFP_10_epochs", "LFP_11_epochs", "LFP_12_epochs", "LFP_13_epochs", "LFP_14_epochs", "LFP_15_epochs", "LFP_16_epochs", "LFP_1_freq", "LFP_1_values", "LFP_2_freq", "LFP_2_values", "LFP_3_freq", "LFP_3_values", "LFP_4_freq", "LFP_4_values", "LFP_5_freq", "LFP_5_values", "LFP_6_freq", "LFP_6_values", "LFP_7_freq", "LFP_7_values", "LFP_8_freq", "LFP_8_values", "LFP_9_freq", "LFP_9_values", "LFP_10_freq", "LF

In [32]:
first_p_off

NameError: name 'first_p_off' is not defined

In [33]:
plot_psd_heatmap(
    np.array(first_p_off["ECOG1_freq"][0]),
    np.array([np.array(arr) for arr in first_p_off["ECOG1_values"][0]]),
)

NameError: name 'first_p_off' is not defined

In [ ]:
plot_avg_psd(
    np.array(first_p_off["ECOG1_freq"][0]),
    np.array([np.array(arr) for arr in first_p_off["ECOG1_values"][0]]),
)

### 4. Behavioral Data Processing


In [ ]:
from utils.motion import tracing_speed

In [ ]:
participants = participants.with_columns(
    pl.struct(["x_interpolated", "y_interpolated", "time_original"])
    .map_elements(
        lambda x: tracing_speed(
            x["x_interpolated"], x["y_interpolated"], x["time_original"]
        ),
        return_dtype=pl.List(pl.Float64),
    )
    .alias("tracing_speed")
)

In [ ]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=1,
    columns=[
        "time_original",
        "tracing_speed",
        "dbs_stim",
        "participant_id",
        "trial",
        "session",
        "block",
    ],
    explode=["time_original", "tracing_speed"],
)

In [ ]:
plot_tracing_speed(trial_df, tracing_speed="tracing_speed", time="time_original")

In [ ]:
participants = participants.with_columns(
    pl.struct(["x", "y", "motion_time"])
    .map_elements(
        lambda x: tracing_speed(x["x"], x["y"], x["motion_time"]),
        return_dtype=pl.List(pl.Float64),
    )
    .alias("original_tracing_speed")
)

In [17]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=1,
    columns=[
        "motion_time",
        "tracing_speed",
        "dbs_stim",
        "participant_id",
        "trial",
        "session",
        "block",
    ],
    explode=["motion_time", "tracing_speed"],
)

In [19]:
plot_tracing_speed(trial_df, time="motion_time")